# Retail Sales Analysis — Exploratory Data Analysis (Pandas + Matplotlib)

**Objective:** Perform end-to-end EDA on a retail sales dataset to demonstrate data cleaning, KPI computation, visualizations, and actionable business insights.

**Dataset expected:** `retail_sales_large.csv` with columns like:
`Order ID, Order Date, Product, Category, Unit Price, Quantity, Total Sales, Customer Type, City, Payment Method`.

This notebook is organized into professional sections: Setup, Load Data, Cleaning, Feature Engineering, EDA, Visualizations, Insights, and Next Steps.


---
## 1) Setup & Imports

Run this cell to import libraries and set plotting defaults.


In [1]:
# Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import os

plt.rcParams.update({'figure.autolayout': True, 'figure.figsize': (10,5)})

# File name (ensure you've uploaded the CSV to Colab files panel)
DATA_FILENAME = 'retail_sales_large.csv'
print('Notebook ready. Dataset file expected:', DATA_FILENAME)

Notebook ready. Dataset file expected: retail_sales_large.csv


---
## 2) Load data

This cell will load the CSV into a DataFrame and display basic info.


In [2]:
# Load dataset
if not os.path.exists(DATA_FILENAME):
    raise FileNotFoundError(f"{DATA_FILENAME} not found. Upload it using the Colab Files panel.")

df = pd.read_csv(DATA_FILENAME)
print('Shape:', df.shape)
display(df.head(10))
display(df.info())

Shape: (5000, 10)


,Order ID,Order Date,Product,Category,Unit Price,Quantity,Total Sales,Customer Type,City,Payment Method
0,ORD101501,2025-01-05 10:45:22,Fitness Tracker Basic,Sports & Fitness,2467,1,2467,New,Kolkata,UPI
1,ORD102586,2025-01-29 05:01:36,Running Shoes,Sports & Fitness,3035,2,6070,Returning,Pune,Cash
2,ORD102653,2023-12-20 17:21:03,Bluetooth Speaker,Electronics,2570,1,2570,Returning,Delhi,Card
3,ORD101055,2025-02-09 13:59:11,Portable SSD 500GB,Electronics,4870,1,4870,Returning,Ahmedabad,Card
4,ORD100705,2024-03-25 11:22:33,Cushion Cover Pack,Home & Kitchen,368,1,368,Returning,Delhi,Netbanking
5,ORD100106,2025-09-09 01:08:37,Gym Gloves,Sports & Fitness,322,4,1288,New,Delhi,Cash
6,ORD100589,2023-08-24 22:50:42,Hand Blender,Home & Kitchen,1917,1,1917,Returning,Mumbai,UPI
7,ORD102468,2025-05-09 10:35:41,Gym Gloves,Sports & Fitness,401,5,2005,Returning,Bengaluru,Card
8,ORD102413,2024-12-27 20:12:34,HD Webcam,Electronics,1509,1,1509,Returning,Bengaluru,Card
9,ORD101600,2024-10-27 02:09:30,Jacket,Clothing,2524,1,2524,Returning,Chennai,Card


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Order ID        5000 non-null   object
 1   Order Date      5000 non-null   object
 2   Product         5000 non-null   object
 3   Category        5000 non-null   object
 4   Unit Price      5000 non-null   int64 
 5   Quantity        5000 non-null   int64 
 6   Total Sales     5000 non-null   int64 
 7   Customer Type   5000 non-null   object
 8   City            5000 non-null   object
 9   Payment Method  5000 non-null   object
dtypes: int64(3), object(7)
memory usage: 390.8+ KB


None

---
## 3) Data Cleaning & Preparation

Steps performed here:
- Normalize column names
- Parse dates
- Ensure numeric columns are numeric
- Compute Total Sales if missing
- Drop rows missing essential info


In [ ]:
# Normalize column names (strip)
df.columns = [c.strip() for c in df.columns]

# Detect key columns (common names)
def find_col(cols, candidates):
    cols_l = {c.lower(): c for c in cols}
    for cand in candidates:
        if isinstance(cand, (list,tuple)):
            for c in cand:
                if c.lower() in cols_l:
                    return cols_l[c.lower()]
        else:
            if cand.lower() in cols_l:
                return cols_l[cand.lower()]
    return None

cols = df.columns.tolist()
date_col = find_col(cols, ['Order Date','order_date','date','orderdate'])
product_col = find_col(cols, ['Product','product','Item','item','sku'])
category_col = find_col(cols, ['Category','category','Product Category','product_category'])
unit_price_col = find_col(cols, ['Unit Price','unit_price','Price','price','UnitPrice'])
qty_col = find_col(cols, ['Quantity','Qty','quantity','qty','Units'])
total_col = find_col(cols, ['Total Sales','Total','Sales','Revenue','total_sales','total'])

print('Detected columns:') 
print('date_col ->', date_col)
print('product_col ->', product_col)
print('category_col ->', category_col)
print('unit_price_col ->', unit_price_col)
print('qty_col ->', qty_col)
print('total_col ->', total_col)

# Parse dates
if date_col:
    df[date_col] = pd.to_datetime(df[date_col], errors='coerce')

# Numeric conversions
for c in [unit_price_col, qty_col, total_col]:
    if c and c in df.columns:
        df[c] = pd.to_numeric(df[c], errors='coerce')

# Fill Quantity with 1 where missing
if qty_col and qty_col in df.columns:
    df[qty_col] = df[qty_col].fillna(1)

# If Total Sales missing, compute from unit_price * qty if possible
if (not total_col) and unit_price_col and qty_col:
    df['Total Sales'] = df[unit_price_col] * df[qty_col]
    total_col = 'Total Sales'
elif total_col and df[total_col].isna().any() and unit_price_col and qty_col:
    mask = df[total_col].isna() & df[unit_price_col].notna() & df[qty_col].notna()
    df.loc[mask, total_col] = df.loc[mask, unit_price_col] * df.loc[mask, qty_col]

# Drop rows missing product or date (not analyzable)
before = len(df)
if product_col and product_col in df.columns:
    df = df[df[product_col].notna()]
if date_col and date_col in df.columns:
    df = df[df[date_col].notna()]
after = len(df)
print(f'Dropped {before-after} rows missing product/date')

display(df.head(5))
display(df.describe(include='all'))

---
## 4) Feature Engineering

Create useful time features and ensure order value metric exists.


In [ ]:
# Year, Month, Year-Month
if date_col:
    df['Year'] = df[date_col].dt.year
    df['Month'] = df[date_col].dt.month
    df['year_month'] = df[date_col].dt.to_period('M').astype(str)

# Ensure Order Value exists
if total_col not in df.columns:
    if unit_price_col and qty_col:
        df['Total Sales'] = df[unit_price_col] * df[qty_col]
        total_col = 'Total Sales'
        
print('Sample feature-engineered columns:') 
display(df[['Year','Month','year_month']].dropna().head())

---
## 5) Exploratory Data Analysis (KPIs and Tables)

Compute key metrics: Total Revenue, Total Orders, Average Order Value, Units Sold, Top products/categories.


In [ ]:
# KPIs
total_revenue = df[total_col].sum() if total_col in df.columns else None
total_orders = df['Order ID'].nunique() if 'Order ID' in df.columns else len(df)
aov = total_revenue / total_orders if total_revenue is not None else None
units_sold = df[qty_col].sum() if qty_col in df.columns else None

print(f'Total Revenue: {total_revenue:,.2f}')
print(f'Total Orders: {total_orders:,}')
print(f'Average Order Value (AOV): {aov:,.2f}')
print(f'Units Sold: {units_sold:,}\n')

# Top 10 products by revenue
if product_col and total_col in df.columns:
    top_products = df.groupby(product_col)[total_col].sum().sort_values(ascending=False).head(10)
    display(top_products.to_frame('Revenue'))

# Sales by category
if category_col and total_col in df.columns:
    sales_by_category = df.groupby(category_col)[total_col].sum().sort_values(ascending=False)
    display(sales_by_category.to_frame('Revenue').head(10))

---
## 6) Visualizations (Matplotlib)

Charts created:
- Sales by Month (line)
- Top 10 Products by Revenue (bar)
- Sales by Category (bar/pie)
- Distribution of Order Values (histogram)

Note: Run all cells and open the 'figures' folder to download PNGs.


In [ ]:
# Make figures directory
fig_dir = 'figures'
os.makedirs(fig_dir, exist_ok=True)

# Sales by month (line)
if 'year_month' in df.columns and total_col in df.columns:
    sales_by_month = df.groupby('year_month')[total_col].sum().sort_index()
    fig, ax = plt.subplots(figsize=(12,5))
    ax.plot(sales_by_month.index, sales_by_month.values, marker='o')
    ax.set_title('Sales by Month')
    ax.set_xlabel('Year-Month')
    ax.set_ylabel('Sales')
    plt.xticks(rotation=45)
    fig_path = os.path.join(fig_dir, 'sales_by_month.png')
    fig.savefig(fig_path)
    plt.show()

# Top 10 products by revenue
if product_col and total_col in df.columns:
    top_products = df.groupby(product_col)[total_col].sum().sort_values(ascending=False).head(10)
    fig, ax = plt.subplots(figsize=(12,5))
    top_products.plot(kind='bar', ax=ax)
    ax.set_title('Top 10 Products by Revenue')
    ax.set_xlabel('Product')
    ax.set_ylabel('Revenue')
    plt.xticks(rotation=45)
    fig_path = os.path.join(fig_dir, 'top_products_revenue.png')
    fig.savefig(fig_path)
    plt.show()

# Sales by category (bar)
if category_col and total_col in df.columns:
    sales_by_category = df.groupby(category_col)[total_col].sum().sort_values(ascending=False)
    fig, ax = plt.subplots(figsize=(10,5))
    sales_by_category.plot(kind='bar', ax=ax)
    ax.set_title('Sales by Category')
    ax.set_xlabel('Category')
    ax.set_ylabel('Revenue')
    plt.xticks(rotation=45)
    fig_path = os.path.join(fig_dir, 'sales_by_category.png')
    fig.savefig(fig_path)
    plt.show()

# Distribution of order values
if total_col in df.columns:
    fig, ax = plt.subplots(figsize=(10,5))
    ax.hist(df[total_col].dropna(), bins=30)
    ax.set_title('Distribution of Order Values')
    ax.set_xlabel('Order Value')
    ax.set_ylabel('Count')
    fig_path = os.path.join(fig_dir, 'order_value_distribution.png')
    fig.savefig(fig_path)
    plt.show()

print('\nAll figures saved in the figures/ folder.')

---
## 7) Insights & Recommendations

Write concise, numbered insights here based on the outputs above. Example template:
1. **Total revenue & trend:** Summarize total revenue and whether sales are growing or seasonal.
2. **Top products:** Mention top 2–3 products by revenue and what to do (stock, promotion).
3. **Category focus:** Which categories drive revenue and which are underperforming.
4. **Customer/payment implications:** If returning customers or a payment method dominates, recommend actions.
5. **Actionable next steps:** e.g., run cohort analysis, A/B test promotions, forecast demand for Q4.


In [ ]:
# Save a short insights.txt automatically (example placeholders)
insights = []
insights.append(f"Total revenue: {total_revenue:,.2f}")
if 'top_products' in globals():
    top_prod = top_products.index[0] if len(top_products)>0 else 'N/A'
    top_val = top_products.iloc[0] if len(top_products)>0 else 0
    insights.append(f"Top product by revenue: {top_prod} (₹{top_val:,.2f})")
if 'sales_by_month' in globals():
    best_month = sales_by_month.idxmax() if len(sales_by_month)>0 else 'N/A'
    insights.append(f"Best month: {best_month}")
insights.append('See figures/ for visualizations.')

with open('insights.txt','w',encoding='utf-8') as f:
    f.write('\n'.join(insights))

print('Saved insights to insights.txt')

---
## Next steps / What to add next (optional)
- Add discounts/returns columns and re-run totals
- Create dashboards (Plotly / Power BI)
- Build forecasting model (Prophet / ARIMA)
- Do customer segmentation and retention analysis
